In [15]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import ipyvolume as ipv



## PLY Header

See also:

http://paulbourke.net/dataformats/ply/

```
ply
format ascii 1.0
element vertex 3090
property float x
property float y
property float z
element face 6871
property list int int vertex_index
property float area
property int epidermis
element edge 9928
property int source
property int target
property list int int face_index
property float length
element volume 32
property list int int face_index
property int label
end_header
```

In [18]:


"""
char       character                 1
uchar      unsigned character        1
short      short integer             2
ushort     unsigned short integer    2
int        integer                   4
uint       unsigned integer          4
float      single-precision float    4
double     double-precision float    8
"""

dtypes = {
    "char": np.uint8,
    "uchar": np.uint8,
    "short": np.int16,
    "ushort": np.uint16,
    "int": np.int32,
    "uint": np.uint32,
    "float": np.float32,
    "double": np.float64,
}

def _parse_header_line(line, element, structures, sizes):
    if line.startswith('element'):
        words = line.split(' ')
        element = words[1]
        structures[element] = {}
        sizes[element] = int(words[2]) 
        
    elif line.startswith('property'):
        words = line.split()
        if "list" in line:
            structures[element][words[-1]] = words[-3:-1]
        else:
            structures[element][words[-1]] = words[-2]
    return element


def parse_header(ply_f):
    structures, sizes = {}, {}
    element = ""
    sizes['header'] = 0
    for line in ply_f:
        sizes['header'] += 1
        if "end_header" in line:
            break
        element = _parse_header_line(
            line,
            element,
            structures, 
            sizes
        )
    else:
        raise IOError('Header end string not found')
    offsets = np.cumsum(list(sizes.values()))
    offsets = {k: v for k, v in zip(list(sizes)[1:], offsets[:-1])}
    
    return structures, sizes, offsets
    
def _parse_line(line, structure):
    words = line.split()
    cursor = 0
    values = {}
    for key, dtype in structure.items():
        if isinstance(dtype, str):
            dtype = dtypes[dtype]
            values[key] = dtype(words[cursor])
            cursor += 1
        elif isinstance(dtype, list):
            dtype = dtypes[dtype[1]]
            length = int(words[cursor])
            cursor += 1
            values[key] = [dtype(w) for w in words[cursor: cursor+length]]
            cursor += length
    return values

def _parse_generic(ply_f, structure, size, offset):
    ply_f.seek(0)
    df = pd.DataFrame(columns=structure.keys())
    for i, line in enumerate(ply_f):
        if i <= offset:
            continue
        if i == size + offset - 1:
            break
        df = df.append(
            _parse_line(line, structure),
            ignore_index=True
        )
    return df.dropna(axis=1, how="all")
        
def _parse_vertex(ply_f, structure, size, offset):
    ply_f.seek(0)
    df = pd.read_csv(
        ply_f,
        nrows=size,
        header=None,
        sep=' ',
        skiprows=offset)
    df.columns = structure.keys()
    
    return df

def _parse_face(ply_f, structure, size, offset):
    """
    Triangular faces
    element face 6871
    property list int int vertex_index
    property float area
    property int epidermis
    """
    ply_f.seek(0)
    df = pd.read_csv(
        ply_f,
        nrows=size,
        sep=' ',
        header=None,
        usecols=[1, 2, 3, 4, 5],
        skiprows=offset
    )
    df.columns = ["v0", "v1", "v2", "area", "epidermis"]
    
    return df


_parsers = {
    "vertex": _parse_vertex,
    "face": _parse_face,
    "volume": _parse_generic
}

with open('data/ply/example_embryo.ply', 'r', encoding='latin_1') as ply_f:
    structures, sizes, offsets = parse_header(ply_f)
    
    datasets = {}
    for element, structure in structures.items():

        parser = _parsers.get(element)
        if parser is None:
            continue
        size = sizes[element]
        offset = offsets[element]
        df = parser(ply_f, structure, size, offset)
        if element == "volume":
            datasets['cell'] = df
            df.index.name = "cell"
        else:
            datasets[element[:4]] = df
            df.index.name = element[:4]
            
            




In [19]:
offsets

{'vertex': 19, 'face': 3109, 'edge': 9980, 'volume': 19908}

In [20]:
cell_df = datasets['cell']

In [21]:
cell_df.shape

(30, 2)

In [22]:
datasets['cell']['num_faces'] = datasets['cell']['face_index'].apply(len)

cell_faces = pd.Series(
    np.concatenate(datasets['cell']['face_index']),
    index=np.repeat(datasets['cell'].index,
                    datasets['cell']['num_faces']),
    name='face'
)


n_cells = cell_faces.value_counts().sort_index()
face_df = datasets['face']

face_df['num_cells'] = 0
face_df.loc[n_cells.index, 'num_cells'] = n_cells

triangles = pd.DataFrame(
    np.repeat(
    face_df[['v0', 'v1', 'v2', 'epidermis']].to_numpy(), 
    face_df['num_cells'], axis=0),
    columns=['v0', 'v1', 'v2', 'epidermis'])
triangles['face'] = np.repeat(
    face_df.index,
    face_df['num_cells']
)
triangles['cell'] = cell_faces.sort_values().index


In [23]:
triangles

,v0,v1,v2,epidermis,face,cell
0,345,1585,2578,1,7,0
1,410,1228,956,1,8,0
2,793,410,1421,1,9,0
3,640,1025,2633,1,10,0
4,2633,1282,2143,1,11,0
5,1819,1030,2143,1,12,0
6,1196,84,1414,1,13,0
7,1144,2115,92,1,14,1
8,672,2050,1460,1,15,1
9,352,2161,1144,1,16,1


In [24]:
vert_df = datasets['vert']

In [25]:

vert_df["valence"] = 0
for v in ['v0', 'v1', 'v2']:
    valence = triangles.groupby(v)['cell'].apply(
        lambda df: df.unique().size
    )
    valence.index.name = 'vert'
    vert_df.loc[valence.index, "valence"] += valence




In [26]:
(vert_df["valence"] > 3).shape

(3090,)

In [190]:
(vert_df["valence"] > 3).shape

(3090,)

In [29]:
border_vs = vert_df[vert_df["valence"] > 4]

border_faces = face_df[face_df[["v0", "v1", "v2"]]
        .isin(border_vs.index).sum(axis=1) > 1]

ipv.clear()

ipv.scatter(
    border_vs.x.to_numpy(),
    border_vs.y.to_numpy(),
    border_vs.z.to_numpy(),
    size=2,
    color='#aa3333',
    marker="sphere"
)
from matplotlib import cm


#"""
ipv.plot_trisurf(
    datasets['vert'].x,
    datasets['vert'].y,
    datasets['vert'].z,
    triangles=triangles[
        ['v0', "v1", 'v2']][
        triangles['epidermis'] == 0
    ],
    color='k'
)
#"""


ipv.squarelim()
ipv.show()

In [242]:
v0s = vert_df[['x', 'y', 'z']].to_numpy().take(face_df["v0"], axis=0) 
v1s = vert_df[['x', 'y', 'z']].to_numpy().take(face_df["v1"], axis=0) 
v2s = vert_df[['x', 'y', 'z']].to_numpy().take(face_df["v2"], axis=0) 

normals = np.cross((v1s - v0s), (v1s - v2s))

In [243]:
v0s.shape

(6871, 3)

In [244]:
face_df.shape

(6871, 6)

In [245]:
normals

array([[ -8.28593979, -19.41802744,   1.86510975],
       [-13.46424589, -35.5913146 ,  -1.76427884],
       [ 18.45014346,  38.17796236,   5.11931688],
       ...,
       [ 13.76464447,  22.28384976,  16.56702238],
       [-10.46983398,   5.96263486,   2.70655844],
       [ 13.18171367,  -7.62772356,  -3.20929534]])

In [130]:
datasets['face'].head()

,v0,v1,v2,area,epidermis,num_cells
face,,,,,,
0,2832,2899,384,10.597113,1,0
1,2512,80,2311,19.046918,1,0
2,136,137,733,21.355163,1,0
3,81,82,709,17.849321,1,0
4,1074,733,1895,18.107296,1,0


In [125]:
datasets['face'].head()

,v0,v1,v2,area,epidermis
face,,,,,
0,2832,2899,384,10.597113,1
1,2512,80,2311,19.046918,1
2,136,137,733,21.355163,1
3,81,82,709,17.849321,1
4,1074,733,1895,18.107296,1


In [95]:
num_faces


cell
0     372
1     208
2     262
3     234
4     556
5     312
6     304
7     314
8     682
9     314
10    304
11    376
12    232
13    394
14    486
15    620
16    332
17    286
18    256
19    314
20    304
21    348
22    218
23    314
24    446
25    282
26    290
27    448
28    416
29    284
Name: face_index, dtype: int64